In [ ]:
DATASET_NAME = "cardiffnlp/tweet_sentiment_multilingual"
MODEL_NAME = "bert-base-multilingual-cased"
CHECKPOINTS_PATH = '/content/drive/MyDrive/model_compression/model2.pt'

# Install requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/NeuroCrushers/model-compression-course

Cloning into 'model-compression-course'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 3), reused 14 (delta 3), pack-reused 0
Receiving objects: 100% (14/14), 28.75 KiB | 9.58 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
%cd model-compression-course

/content/model-compression-course


In [ ]:
!pip install -r requirements.txt

  Using cached datasets-2.14.5-py3-none-any.whl (519 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached transformers-4.33.2-py3-none-any.whl (7.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00


# Load data

In [ ]:
from datasets import load_dataset

dataset = load_dataset(DATASET_NAME, "all")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
RANDOM_SEED = 42
dataset = dataset.shuffle(seed=RANDOM_SEED)
dataset.set_format(type="torch", columns=["text", "label"])

In [ ]:
NUM_SHARDS = 1
train_dataset = dataset['train'].shard(num_shards=NUM_SHARDS, index=0)
val_dataset = dataset['validation'].shard(num_shards=NUM_SHARDS, index=0)
test_dataset = dataset['test'].shard(num_shards=NUM_SHARDS, index=0)

In [ ]:
id2label = {2: "positive", 1: "neutral", 0:"negative"}
label2id = {"positive" : 2, "neutral": 1, "negative" : 0}

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from tqdm.notebook import tqdm

train_text_lengths = []
for text in tqdm(dataset['train']['text']):
    train_text_lengths.append(len(tokenizer(text)[0]))
print(f'Average text length: {np.mean(train_text_lengths)}')
print(f'Median text length: {np.median(train_text_lengths)}')
print(f'99 quantile: {np.quantile(train_text_lengths, 0.99)}')
print(f'Max text length: {max(train_text_lengths)}')

  0%|          | 0/14712 [00:00<?, ?it/s]

Average text length: 29.46037248504622
Median text length: 29.0
99 quantile: 63.0
Max text length: 274


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load(CHECKPOINTS_PATH, map_location = device)

# Code for training

In [ ]:
import gc
def cleanup():
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
def tokenize(batch):
        return tokenizer(batch["text"], return_tensors="pt",  truncation=True, padding=True, max_length=65)

In [ ]:
import os
import sys
import torch
from torch.optim import AdamW, Adam
from torch.nn import CrossEntropyLoss, BCELoss
from torch.nn.functional import softmax
from tqdm.notebook import tqdm


class Trainer:
    def __init__(self, model, device, learning_rate = 1e-4, weight_decay = 1e-3, path_to_weights='/content/drive/MyDrive/model_compression/model.pt'):
        self.device = device
        self.model = model.to(self.device)
        self.optimizer = Adam(list(self.model.parameters()), lr=learning_rate, weight_decay=weight_decay)
        self.criterion = CrossEntropyLoss()
        self.path_to_weights = path_to_weights

    def train(self, train_loader, val_loader, num_epochs, save_model=False):
        loss_history = []
        train_history = []
        val_history = []
        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}')
            self.model.train()
            loss_accum = 0
            correct = 0
            total = 0
            for batch_num, batch in enumerate(tqdm(train_loader), start=1):
                prediction, y = self.get_prediction(batch)
                loss = self.compute_loss(prediction, y)
                loss_accum += loss
                cleanup()
                accuracy, correct, total = self.compute_accuracy(prediction, y, correct, total)
                sys.stdout.write(f'\rLoss: {loss}. Train accuracy: {accuracy}')
                sys.stdout.flush()

            mean_loss = loss_accum / batch_num
            train_accuracy = float(correct) / total
            val_accuracy = self.compute_val_accuracy(val_loader)

            loss_history.append(float(mean_loss))
            train_history.append(train_accuracy)
            val_history.append(val_accuracy)

            print(
               f"\nAverage loss: {mean_loss}, Train accuracy: {train_accuracy}, Val accuracy: {val_accuracy}" )
            if save_model:
                self.save_model()
        val_accuracy = self.compute_val_accuracy(val_loader)
        return loss_history, train_history, val_history

    def get_prediction(self, batch):
        tokenized_batch = tokenize(batch)
        ids = tokenized_batch['input_ids'].to(self.device)
        mask = tokenized_batch['attention_mask'].to(self.device)
        y = batch['label'].to(self.device)
        prediction = self.model(input_ids=ids, attention_mask=mask).logits
        return prediction, y

    def compute_loss(self, prediction, y):
        loss = self.criterion(prediction, y)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.item()

    def compute_accuracy(self, prediction, y_true, correct, total):
        y_pred = torch.argmax(prediction, axis=1)
        correct += torch.sum(y_pred == y_true)
        total += y_true.shape[0]
        accuracy = correct / total
        return accuracy, correct, total

    def compute_val_accuracy(self, val_loader):
        self.model.eval()
        correct = 0
        total = 0
        for batch_num, batch in enumerate(tqdm(val_loader)):
            prediction, y = self.get_prediction(batch)
            accuracy, correct, total = self.compute_accuracy(prediction, y, correct, total)
            sys.stdout.write(f'\rVal accuracy: {accuracy}')
            sys.stdout.flush()
        accuracy = float(correct) / total
        return accuracy

    def save_model(self):
        torch.save(self.model, self.path_to_weights)
        print(f'Model saved to {self.path_to_weights}')

# Training

In [ ]:
trainer = Trainer(model, device, learning_rate = 1e-5, weight_decay = 1e-2, path_to_weights = CHECKPOINTS_PATH)

In [ ]:
trainer.train(train_loader, val_loader, num_epochs=1, save_model=True)

Epoch 1


  0%|          | 0/460 [00:00<?, ?it/s]

Loss: 0.24475009739398956. Train accuracy: 0.910073459148407

  0%|          | 0/81 [00:00<?, ?it/s]

Val accuracy: 0.5304784178733826
Average loss: 0.2401664846698227, Train accuracy: 0.9100734094616639, Val accuracy: 0.5304783950617284
Model saved to /content/drive/MyDrive/model_compression/model2.pt


  0%|          | 0/81 [00:00<?, ?it/s]

Val accuracy: 0.5304784178733826

([0.2401664846698227], [0.9100734094616639], [0.5304783950617284])